In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('data/spotify_millsongdata.csv', sep=',')
df.head()

import timeit

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [8]:
print(df['text'].apply(lambda x: len(x.split(' '))).mean())
print(df['text'].apply(lambda x: len(x.split(' '))).std())
print(df['text'].apply(lambda x: len(x.split(' '))).max())
print(df['text'].apply(lambda x: len(x.split(' '))).min())

263.76199479618384
127.15808413527363
950
63


In [5]:
# drop all data where the artist has less than 30 songs
df = df.groupby('artist').filter(lambda x: len(x) > 30)
# remove punctuation
df['text'] = df['text'].str.replace('[^\w\s]','')
# remove numbers
df['text'] = df['text'].str.replace('\d+', '')
# remove all articles
df['text'] = df['text'].str.replace(r'\b(a|an|the)\b', '')
# from nltk.corpus import stopwords
# import nltk
# nltk.download('stopwords')
# stop = stopwords.words('english')
# df['text'] = df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

# prepare text data for a recurrent network
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.utils import to_categorical

# create a tokenizer
t = Tokenizer()
t.fit_on_texts(df['text'])
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(df['text'])
# pad documents to be as long as the longest sequence in the dataset
max_length = df['text'].apply(lambda x: len(x.split(' '))).max()
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

C:\Users\joren\AppData\Local\Temp\ipykernel_9296\1605716740.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('[^\w\s]','')
C:\Users\joren\AppData\Local\Temp\ipykernel_9296\1605716740.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('\d+', '')
C:\Users\joren\AppData\Local\Temp\ipykernel_9296\1605716740.py:8: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace(r'\b(a|an|the)\b', '')


In [14]:
# prepare labels
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(df['artist'])
# binary encode
onehot_encoded = to_categorical(integer_encoded)

In [16]:
# split data into train and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(padded_docs, onehot_encoded, test_size=0.2, random_state=42)

In [17]:
# define the model as an GRU network
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import GRU
from keras.layers import Embedding

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_length))
model.add(GRU(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))

model.add(Dense(onehot_encoded.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 806, 100)          9038900   
                                                                 
 gru (GRU)                   (None, 100)               60600     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 100)               10100     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 525)               53025     
                                                                 
Total params: 9,182,825
Trainable params: 9,182,825
Non-

In [19]:
# fit the model using a graphics card
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [checkpoint, earlystop]
# fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, callbacks=callbacks_list)



Epoch 1/50
 27/697 [>.............................] - ETA: 56:06 - loss: 6.2230 - accuracy: 0.0052

KeyboardInterrupt: 